In [8]:
import json
f = open('./1.json')
json_file = json.load(f)

In [9]:
json_file[1]

{'id': 'Aq6nkgCbWc',
 'type': 'polygonlabels',
 'value': {'points': [[3.4166737705501578, 53.19465654365013],
   [5.256421185461781, 52.75703235958848],
   [7.818926513374399, 51.706734317840485],
   [9.724379193104296, 50.656436276092485],
   [11.038484489469742, 52.75703235958848],
   [12.352589785835185, 54.1574297485858],
   [15.177916173020892, 58.4461467523901],
   [13.338168758109267, 59.058820610076424],
   [11.3670108135611, 60.721792509510735],
   [10.118610782013926, 62.90991342981906],
   [9.33014760419466, 65.18555918693971],
   [9.33014760419466, 68.16140363855902],
   [9.461558133831202, 70.96219841655366],
   [5.519242244734871, 69.91190037480568],
   [4.336547478005969, 68.86160233305768],
   [3.6794948298232466, 66.2358572286877],
   [3.1538527112770685, 63.52258728750539],
   [2.431094798276074, 61.07189185676008],
   [1.8397474149116233, 58.35862191557776],
   [2.233979003821257, 55.90792648483245]],
  'polygonlabels': ['Bumper']},
 'to_name': 'image',
 'from_name':

In [10]:
def getboundingbox(points) :
    x_array  = [x[0] for x in points]
    y_array = [x[1] for x in points]
    start_point = (int(min(x_array)), int(min(y_array)))
    end_point = (int(max(x_array)),int(max(y_array)))
    return [start_point,end_point]

In [11]:
def unnormalize_coordinates(points,width,height) :
    new_points = []
    for point in points :
        x_new = (point[0] * width)/100
        y_new = (point[1] * height)/100
        temp_point = [x_new,y_new]
        new_points.append(temp_point)
    return new_points
        
    
    

In [12]:
def plot_rectangle(points,image) :
    
    start_point = points[0]

    # Ending coordinate, here (220, 220)
    # represents the bottom right corner of rectangle
    end_point = points[1]

    # Blue color in BGR
    color = (0,255,0)

    # Line thickness of 2 px
    thickness = 4

    # Using cv2.rectangle() method
    # Draw a rectangle with blue line borders of thickness of 2 px
    image = cv2.rectangle(image, start_point, end_point, color, thickness)
    
    
    return image

In [13]:
import numpy as np
def plot_on_image(image,normalized_points) :
    pts = np.array(normalized_points)
    pts = pts.reshape((-1, 1, 2))
    color = (0, 255, 0)
    isClosed = True
    # Line thickness of 8 px
    thickness = 4
    shapes = np.zeros_like(image, np.uint8)
    cv2.fillPoly(shapes, np.int32([pts]), color=(0, 255, 0))
    out = image.copy()
    alpha = 0.5
    mask = shapes.astype(bool)
    out[mask] = cv2.addWeighted(image, alpha, shapes, 1 - alpha, 0)[mask]
    return out
    
    

In [14]:
image_folder_name = './images'
json_folder_name = './data'
import os
import cv2
files = os.listdir(image_folder_name)
for file in files :
    image = cv2.imread(os.path.join(image_folder_name,file))
    name = file.split('.')[0]
    f = open(os.path.join(json_folder_name,name+'.json'))
    json_file = json.load(f)
    for item in json_file :
        if item['type'] == 'polygonlabels' :
            points = item['value']['points']
            width = item['original_width']
            height = item['original_height']
            unnormalized_points = unnormalize_coordinates(points,width,height)
            image = plot_on_image(image,unnormalized_points)
            bb_cords = getboundingbox(unnormalized_points)
            image = plot_rectangle(bb_cords,image)
            

    while(1):
        cv2.imshow('img',image)
        k = cv2.waitKey(33)
        if k==27:    # Esc key to stop
            break
            
    